# <a id='data'>1. Data Objective</a>

The objective of the M5 forecasting competition is to advance the theory and practice of forecasting by identifying the method(s) that provide the most accurate point forecasts for each of the 42,840 time series of the competition. I addition, to elicit information to estimate the uncertainty distribution of the realized values of these series as precisely as possible. 

To that end, the participants of M5 are asked to provide 28 days ahead point forecasts (PFs) for all the series of the competition, as well as the corresponding median and 50%, 67%, 95%, and 99% prediction intervals (PIs).
The M5 differs from the previous four ones in five important ways, some of them suggested by the discussants of the M4  competition, as follows:

- First, it uses grouped unit sales data, starting at the product-store level and being aggregated to that of product departments, product categories, stores, and three geographical areas: the States of California (CA), Texas (TX), and Wisconsin (WI).

- Second, besides the time series data, it includes explanatory variables such as sell prices, promotions, days of the week, and special events (e.g. Super Bowl, Valentine’s Day, and Orthodox Easter) that typically affect unit sales and could improve forecasting accuracy.

- Third, in addition to point forecasts, it assesses the distribution of uncertainty, as the participants are asked to provide information on nine indicative quantiles.

- Fourth, instead of having a single competition to estimate both the point forecasts and the uncertainty distribution, there will be two parallel tracks using the same dataset, the first requiring 28 days ahead point forecasts and the second 28 days ahead probabilistic forecasts for the median and four prediction intervals (50%, 67%, 95%, and 99%).

- Fifth, for the first time it focuses on series that display intermittency, i.e., sporadic demand including zeros.


### 1.1. Dates and hosting

The M5 will start on March 2, 2020 and finish on June 30 of the same year. The competition will be run using the Kaggle platform. Thus, we expect many submissions from all types of forecasters including data scientists, statisticians, and practitioners, expanding the field of forecasting and eventually integrating its various approaches for improving accuracy and uncertainty estimation.

The competition will be divided into two separate Kaggle competitions, using the same dataset, with the first (M5 Forecasting Competition – Accuracy) requiring 28 days ahead point forecasts and the second (M5 Forecasting Competition – Uncertainty) 28 days ahead probabilistic forecasts for the corresponding median and four prediction intervals (50%, 67%, 95%, and 99%).


### 1.2. The dataset

The M5 dataset, generously made available by Walmart, involves the unit sales of various products sold in the USA, organized in the form of grouped time series. More specifically, the dataset involves the unit sales of 3,049 products, classified in 3 product categories (Hobbies, Foods, and Household) and 7 product departments, in which the above-mentioned categories are disaggregated.  The products are sold across ten stores, located in three States (CA, TX, and WI). In this respect, the bottom-level of the hierarchy, i.e., product-store unit sales can be mapped across either product categories or geographical regions

![Picture1.png](./Picture1.png)

The historical data range from 2011-01-29 to 2016-06-19. Thus, the products have a (maximum) selling history of 1,941  days / 5.4 years (test data of h=28 days not included). 

### The M5 dataset consists of the following four (4) files:

### File 1: “calendar.csv” 
Contains information about the dates the products are sold.
- date: The date in a “y-m-d” format.
- wm_yr_wk: The id of the week the date belongs to.
- weekday: The type of the day (Saturday, Sunday, …, Friday).
- wday: The id of the weekday, starting from Saturday.
- month: The month of the date.
- year: The year of the date.
- event_name_1: If the date includes an event, the name of this event.
- event_type_1: If the date includes an event, the type of this event.
- event_name_2: If the date includes a second event, the name of this event.
- event_type_2: If the date includes a second event, the type of this event.
- snap_CA, snap_TX, and snap_WI: A binary variable (0 or 1) indicating whether the stores of CA, TX or WI allow SNAP  purchases on the examined date. 1 indicates that SNAP purchases are allowed.

### File 2: “sell_prices.csv”
Contains information about the price of the products sold per store and date.
- store_id: The id of the store where the product is sold. 
- item_id: The id of the product.
- wm_yr_wk: The id of the week.
- sell_price: The price of the product for the given week/store. The price is provided per week (average across seven days). If not available, this means that the product was not sold during the examined week. Note that although prices are constant at weekly basis, they may change through time (both training and test set).  

### File 3: “sales_train_valid.csv” 
Contains the historical daily unit sales data per product and store.
- item_id: The id of the product.
- dept_id: The id of the department the product belongs to.
- cat_id: The id of the category the product belongs to.
- store_id: The id of the store where the product is sold.
- state_id: The State where the store is located.
- d_1, d_2, …, d_i, … d_1941: The number of units sold at day i, starting from 2011-01-29.

### File 4: “sales_train_evaluation.csv” 
Contains the historical daily unit sales data per product and store.
- item_id: The id of the product.
- dept_id: The id of the department the product belongs to.
- cat_id: The id of the category the product belongs to.
- store_id: The id of the store where the product is sold.
- state_id: The State where the store is located.
- d_1, d_2, …, d_i, … d_1969: The number of units sold at day i, starting from 2011-01-29. 


# <a id='fetch'>2. Fetch the data</a>
### 2.1. Import libraries

In [ ]:
import os, gc

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error

import lightgbm as lgb

import plotly_express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
for dirname, _, filenames in os.walk('data/m5-forecasting-accuracy/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

### 2.2. Setting working directory and loading datasets

In [ ]:
# working path
path = 'data/m5-forecasting-accuracy/'

In [ ]:
# loading datasets
calendar = pd.read_csv(path + 'calendar.csv')
sellPrice = pd.read_csv(path + 'sell_prices.csv')
sales = pd.read_csv(path + 'sales_train_evaluation.csv')
sampleSubmission = pd.read_csv(path + 'sample_submission.csv')

In [ ]:
print('Calendar dataset has {} rows and {} columns'.format(calendar.shape[0], calendar.shape[1]))
print('Sell Price dataset has {} rows and {} columns'.format(sellPrice.shape[0], sellPrice.shape[1]))
print('Sales dataset has {} rows and {} columns'.format(sales.shape[0], sales.shape[1]))
print('Sample Submission dataset has {} rows and {} columns'.format(sampleSubmission.shape[0], sampleSubmission.shape[1]))

In [ ]:
# Function using to get number of rows and data types
def infoData(df):
    columns = []
    values = []
    for feature in df.columns:
        columns.append(feature)
        values.append(len(df[feature].unique()))
    percent_missing = df.isnull().sum() * 100 / len(df)
    data = {'Unique_Value': values,'Percent_Missing': percent_missing, 'Data_Types': df.dtypes.values}
    return pd.DataFrame(data=data)

In [ ]:
# Downcast in order to save memory
def reduce_mem_usage(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024**2 
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i,t in enumerate(types):
        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        elif t == np.object:
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
# reducing memory of datasets
calendar = reduce_mem_usage(calendar)
sellPrice = reduce_mem_usage(sellPrice)
sales = reduce_mem_usage(sales)

In [ ]:
calendar.head()

In [ ]:
sellPrice.head()

In [ ]:
sales.head()

Validation data is now from 1914 to 1941. And test data is from 1942 to 1969

In [ ]:
# adding more days to use as test datasets
for d in range(1942, 1970):
    col = 'd_' + str(d)
    sales[col] = 0
    sales[col] = sales[col].astype(np.int16)

In [ ]:
sales.head()

# <a id='melt'>3. Melting the data</a>

In this case what the melt function is doing is that it is converting the sales dataframe which is in wide format to a long format. I have kept the id variables as id, item_id, dept_id, cat_id, store_id and state_id. They have in total 30490 unique values when compunded together. Now the total number of days for which we have the data is 1969 days. Therefore the melted dataframe will be having 30490x1969 i.e. 60034810 rows

In [ ]:
identifierVariables = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
variableName = 'd'
measuredVariables = 'sales'

data = pd.melt(sales,
               id_vars = identifierVariables,
               var_name = variableName,
               value_name = measuredVariables).drop_duplicates()

del identifierVariables, variableName, measuredVariables
gc.collect()
print('Dataset has {} rows and {} columns'.format(data.shape[0], data.shape[1]))

In [ ]:
data.head()

Combine price data from prices dataframe and days data from calendar dataset.

In [ ]:
data = data.merge(calendar, how='left', on='d')
data = data.merge(sellPrice, on=['store_id', 'item_id', 'wm_yr_wk'], how='left')

del calendar, sellPrice
gc.collect()
print('Dataset has {} rows and {} columns'.format(data.shape[0], data.shape[1]))

In [ ]:
data.head()

# <a id='eda'>4. Exploratory Data Analysis</a>
### 4.1 The Dataset

In [ ]:
group = sales.groupby(['state_id','store_id','cat_id','dept_id'],as_index=False)['item_id'].count().dropna()
group['USA'] = 'United States of America'
group.rename(columns={'state_id':'State','store_id':'Store','cat_id':'Category','dept_id':'Department','item_id':'Count'}, 
             inplace=True)
fig = px.treemap(group, path=['USA', 'State', 'Store', 'Category', 'Department'], values='Count',
                  color='Count',
                  color_continuous_scale= px.colors.sequential.Sunset,
                  title='Walmart: Distribution of items')
fig.update_layout(template='seaborn')
fig.show()

### 4.2 Item Prices

In [ ]:
group_price_store = data.groupby(['state_id','store_id','item_id'], as_index=False)['sell_price'].mean().dropna()
fig = px.violin(group_price_store, x='store_id', color='state_id', y='sell_price', box=True, hover_name='item_id')
fig.update_xaxes(title_text='Store')
fig.update_yaxes(title_text='Selling Price($)')
fig.update_layout(template='seaborn', title='Distribution of Items prices wrt Stores', legend_title_text='State')
fig.show()

In [ ]:
group_price_cat = data.groupby(['store_id', 'cat_id', 'item_id'],as_index=False)['sell_price'].mean().dropna()
fig = px.violin(group_price_cat, x='store_id', color='cat_id', y='sell_price', box=True, hover_name='item_id')
fig.update_xaxes(title_text='Store')
fig.update_yaxes(title_text='Selling Price($)')
fig.update_layout(template='seaborn',title='Distribution of Items prices wrt Stores across Categories',
                 legend_title_text='Category')
fig.show()

As can be seen from the plot above, food category items are quite cheap as compared with hobbies and household items. Hobbies and household items have almost the same price range.

### 4.3 Items Sales

In [ ]:
group = data.groupby(['year','date', 'state_id', 'store_id'], as_index=False)['sales'].sum().dropna()
fig = px.violin(group, x='store_id', y='sales', box=True)
fig.update_xaxes(title_text='Store')
fig.update_yaxes(title_text='Total items sales')
fig.update_layout(template='seaborn', title='Distribution of Items Sales wrt Stores', legend_title_text='State')
fig.show()

- California: CA_3 has sold the most number of items while, CA_4 has sold the least number of items.
- Texas: TX_2 and TX_3 have sold the maximum number of items. TX_1 has sold the least number of items.
- Wisconsin: WI_2 has sold the maximum number of items while, WI_3 has sold the least number of items.
- USA: CA_3 has sold the most number of items while, CA_4 has sold the least number of items.

In [ ]:
fig = go.Figure()
title = 'Items Sales Over Time'
years = group.year.unique().tolist()
buttons = []
y=3
for state in group.state_id.unique().tolist():
    group_state = group[group['state_id']==state]
    for store in group_state.store_id.unique().tolist():
        group_state_store = group_state[group_state['store_id']==store]
        fig.add_trace(go.Scatter(name=store, x=group_state_store['date'], y=group_state_store['sales'], showlegend=True, 
                                   yaxis='y'+str(y) if y!=1 else 'y'))
    y-=1

fig.update_layout(
        xaxis=dict(
        #autorange=True,
        range = ['2011-01-29','2016-05-22'],
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(count=2,
                     label="2y",
                     step="year",
                     stepmode="backward"),
                dict(count=3,
                     label="3y",
                     step="year",
                     stepmode="backward"),
                dict(count=4,
                     label="4y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            autorange=True,
        ),
        type="date"
    ),
    yaxis=dict(
        anchor="x",
        autorange=True,
        domain=[0, 0.33],
        mirror=True,
        showline=True,
        side="left",
        tickfont={"size":10},
        tickmode="auto",
        ticks="",
        title='WI',
        titlefont={"size":20},
        type="linear",
        zeroline=False
    ),
    yaxis2=dict(
        anchor="x",
        autorange=True,
        domain=[0.33, 0.66],
        mirror=True,
        showline=True,
        side="left",
        tickfont={"size":10},
        tickmode="auto",
        ticks="",
        title = 'TX',
        titlefont={"size":20},
        type="linear",
        zeroline=False
    ),
    yaxis3=dict(
        anchor="x",
        autorange=True,
        domain=[0.66, 1],
        mirror=True,
        showline=True,
        side="left",
        tickfont={"size":10},
        tickmode="auto",
        ticks='',
        title="CA",
        titlefont={"size":20},
        type="linear",
        zeroline=False
    )
    )
fig.update_layout(template='seaborn', title=title)
fig.show()

### 4.4 State wise Analysis

Let's see sales and revenue of all the stores individually across all the three states: California, Texas & Wisconsin. Plotting total three plots for each store: CA_1, CA_2, CA_3, CA_4, TX_1, TX_2, TX_3, WI_1, WI_2 & WI_3. Details about the plots are as follows:

- First plot shows the daily sales of a store, plotting the values separately for SNAP days. Also, SNAP promotes food purchase, and food sales as well to check if it really affects the food sales.
- Second plot shows the daily revenue of a store with separate plotting for <a href='#SNAP'>SNAP</a> days.
- Third is a heatmap to show daily sales. It's plotted in such a way that it becomes easier to see day wise values.

<div class="alert alert-info" role="alert">
<a id='SNAP'><b>What is SNAP?</b></a><br>
The United States federal government provides a nutrition assistance benefit called the Supplement Nutrition Assistance Program (SNAP).  SNAP provides low income families and individuals with an Electronic Benefits Transfer debit card to purchase food products.  In many states, the monetary benefits are dispersed to people across 10 days of the month and on each of these days 1/10 of the people will receive the benefit on their card.  More information about the SNAP program can be found [here.](https://www.fns.usda.gov/snap/supplemental-nutrition-assistance-program)
</div>

<div class="alert alert-danger" role="alert">
For the heatmaps, the data is till 16th week of 2016 and datetime.weekofyear of function is returning 1,2 & 3 january of 2016 in 53rd week. Plotly's heatmap is connecting the data gap between the 16th and 53rd week. Still figuring out on how to remove this gap.
</div>

In [ ]:
data['revenue'] = data['sales'] * data['sell_price'].astype(np.float32)

In [ ]:
def introduce_nulls(df):
    idx = pd.date_range(df.date.dt.date.min(), df.date.dt.date.max())
    df = df.set_index('date')
    df = df.reindex(idx)
    df.reset_index(inplace=True)
    df.rename(columns={'index':'date'},inplace=True)
    return df

def plot_metric(df,state,store,metric):
    store_sales = df[(df['state_id']==state)&(df['store_id']==store)&(df['date']<='2016-05-22')]
    food_sales = store_sales[store_sales['cat_id']=='FOODS']
    store_sales = store_sales.groupby(['date','snap_'+state],as_index=False)['sales', 'revenue'].sum()
    snap_sales = store_sales[store_sales['snap_'+state]==1]
    non_snap_sales = store_sales[store_sales['snap_'+state]==0]
    food_sales = food_sales.groupby(['date','snap_'+state],as_index=False)['sales', 'revenue'].sum()
    snap_foods = food_sales[food_sales['snap_'+state]==1]
    non_snap_foods = food_sales[food_sales['snap_'+state]==0]
    non_snap_sales = introduce_nulls(non_snap_sales)
    snap_sales = introduce_nulls(snap_sales)
    non_snap_foods = introduce_nulls(non_snap_foods)
    snap_foods = introduce_nulls(snap_foods)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=non_snap_sales['date'],y=non_snap_sales[metric],
                           name='Total '+metric+'(Non-SNAP)'))
    fig.add_trace(go.Scatter(x=snap_sales['date'],y=snap_sales[metric],
                           name='Total '+metric+'(SNAP)'))
    fig.add_trace(go.Scatter(x=non_snap_foods['date'],y=non_snap_foods[metric],
                           name='Food '+metric+'(Non-SNAP)'))
    fig.add_trace(go.Scatter(x=snap_foods['date'],y=snap_foods[metric],
                           name='Food '+metric+'(SNAP)'))
    fig.update_yaxes(title_text='Total items sold' if metric=='sold' else 'Total revenue($)')
    fig.update_layout(template='seaborn',title=store)
    fig.update_layout(
        xaxis=dict(
        #autorange=True,
        range = ['2011-01-29','2016-05-22'],
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(count=2,
                     label="2y",
                     step="year",
                     stepmode="backward"),
                dict(count=3,
                     label="3y",
                     step="year",
                     stepmode="backward"),
                dict(count=4,
                     label="4y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            autorange=True,
        ),
        type="date"
    ))
    return fig

In [ ]:
cal_data = group.copy()
cal_data = cal_data[cal_data.date <= '22-05-2016']
cal_data['week'] = cal_data.date.dt.weekofyear
cal_data['day_name'] = cal_data.date.dt.day_name()

In [ ]:
def calmap(cal_data, state, store, scale):
    cal_data = cal_data[(cal_data['state_id']==state)&(cal_data['store_id']==store)]
    years = cal_data.year.unique().tolist()
    fig = make_subplots(rows=len(years),cols=1,shared_xaxes=True,vertical_spacing=0.005)
    r=1
    for year in years:
        data = cal_data[cal_data['year']==year]
        data = introduce_nulls(data)
        fig.add_trace(go.Heatmap(
            z=data.sales,
            x=data.week,
            y=data.day_name,
            hovertext=data.date.dt.date,
            coloraxis = "coloraxis",name=year,
        ),r,1)
        fig.update_yaxes(title_text=year,tickfont=dict(size=5),row = r,col = 1)
        r+=1
    fig.update_xaxes(range=[1,53],tickfont=dict(size=10), nticks=53)
    fig.update_layout(coloraxis = {'colorscale':scale})
    fig.update_layout(template='seaborn', title=store)
    return fig

<a id="cal" class="btn btn-primary btn-lg btn-block active" role="button" aria-pressed="true" style="color:white;">California</a>
<img src='https://www.pixel4k.com/wp-content/uploads/2018/09/san-francisco-california-cityscape-4k_1538070292.jpg'
style="width:800px;height:400px;">

### CA_1

In [ ]:
fig = plot_metric(data,'CA','CA_1','sales')
fig.show()

In [ ]:
fig = plot_metric(data,'CA','CA_1','revenue')
fig.show()

In [ ]:
fig = calmap(cal_data, 'CA', 'CA_1', 'magma')
fig.show()

### CA_2</h2>

In [ ]:
fig = plot_metric(data, 'CA', 'CA_2', 'sales')
fig.show()

In [ ]:
fig = plot_metric(data, 'CA', 'CA_2', 'revenue')
fig.show()

In [ ]:
fig = calmap(cal_data, 'CA', 'CA_2', 'magma')
fig.show()

### CA_3

In [ ]:
fig = plot_metric(data,'CA','CA_3','sales')
fig.show()

In [ ]:
fig = plot_metric(data,'CA','CA_3','revenue')
fig.show()

In [ ]:
fig = calmap(cal_data, 'CA', 'CA_3', 'magma')
fig.show()

### CA_4

In [ ]:
fig = plot_metric(data, 'CA', 'CA_4', 'sales')
fig.show()

In [ ]:
fig = plot_metric(data,'CA','CA_4','revenue')
fig.show()

In [ ]:
fig = calmap(cal_data, 'CA', 'CA_4', 'magma')
fig.show()

<a id="tex" class="btn btn-primary btn-lg btn-block active" role="button" aria-pressed="true" style="color:white;">Texas</a>
<img src='https://wallpaperaccess.com/full/227248.jpg'
style="width:800px;height:300px;">

### TX_1

In [ ]:
fig = plot_metric(data,'TX','TX_1','sales')
fig.show()

In [ ]:
fig = plot_metric(data,'TX','TX_1','revenue')
fig.show()

In [ ]:
fig = calmap(cal_data, 'TX', 'TX_1', 'viridis')
fig.show()

### TX_2

In [ ]:
fig = plot_metric(data,'TX','TX_2','sales')
fig.show()

In [ ]:
fig = plot_metric(data,'TX','TX_2','revenue')
fig.show()

In [ ]:
fig = calmap(cal_data, 'TX', 'TX_2', 'viridis')
fig.show()

### TX_3

In [ ]:
fig = plot_metric(data,'TX','TX_3','sales')
fig.show()

In [ ]:
fig = plot_metric(data,'TX','TX_3','revenue')
fig.show()

In [ ]:
fig = calmap(cal_data, 'TX', 'TX_3', 'viridis')
fig.show()

<a id="wis" class="btn btn-primary btn-lg btn-block active" role="button" aria-pressed="true" style="color:white;">Wisconsin</a>
<img src='https://i.ytimg.com/vi/RzETB_wVAKI/maxresdefault.jpg'
style="width:800px;height:300px;">

### WI_1

In [ ]:
fig = plot_metric(data,'WI','WI_1','sales')
fig.show()

In [ ]:
fig = plot_metric(data,'WI','WI_1','revenue')
fig.show()

In [ ]:
fig = calmap(cal_data, 'WI', 'WI_1', 'twilight')
fig.show()

### WI_2

In [ ]:
fig = plot_metric(data,'WI','WI_2','sales')
fig.show()

In [ ]:
fig = plot_metric(data, 'WI','WI_2','revenue')
fig.show()

In [ ]:
fig = calmap(cal_data, 'WI', 'WI_2', 'twilight')
fig.show()

### WI_3


In [ ]:
fig = plot_metric(data,'WI','WI_3','sales')
fig.show()

In [ ]:
fig = plot_metric(data,'WI','WI_3','revenue')
fig.show()

In [ ]:
fig = calmap(cal_data, 'WI', 'WI_3', 'twilight')
fig.show()

In [ ]:
del group, group_price_cat, group_price_store, group_state, group_state_store, cal_data
gc.collect();

# <a id='fe'>5. Feature Engineering</a>

<img src='https://media.giphy.com/media/yv1ggi3Cbase05a8iS/giphy.gif' style="width:500px;height:300px;">

The goal of feature engineering is to provide strong and ideally simple relationships between new input features and the output feature for the supervised learning algorithm to model.

### 5.1 Label Encoding
<ol>
  <li>Remove unwanted data to create space in RAM for further processing.</li>
  <li>Label Encode categorical features.(I had converted already converted categorical variable to category type. So, I can simply use their codes instead of using LableEncoder)</li>
  <li>Remove date as its features are already present.</li>
</ol>

In [ ]:
nanFeatures = ['event_name_1', 'event_name_2', 'event_type_1', 'event_type_2']
for feature in nanFeatures:
    data[feature].fillna('MISSING', inplace=True)
    
del nanFeatures
gc.collect()
data

In [ ]:
categorical_features = data.columns.drop([
    'sales', 'date', 'sell_price', 'wm_yr_wk', 'snap_CA', 'snap_TX', 'snap_WI', 'id', 'd'])

le = LabelEncoder()
for feature in categorical_features:
    data[feature] = le.fit_transform(data[feature])

del categorical_features
gc.collect()
data

### 5.2 Introduce Lags

Lag features are the classical way that time series forecasting problems are transformed into supervised learning problems.

Introduce lags to the the target variable `sales`. The maximum lag I have introduced is 30 days. It's purely upto you how many lags you want to introduce.

In [ ]:
data['lag_t28'] = data.groupby(['id'])['sales'].transform(lambda x: x.shift(28)).astype(np.float16)
data['lag_t29'] = data.groupby(['id'])['sales'].transform(lambda x: x.shift(29)).astype(np.float16)
data['lag_t30'] = data.groupby(['id'])['sales'].transform(lambda x: x.shift(30)).astype(np.float16)

### 5.3 Mean and Median Encoding

From a mathematical point of view, mean encoding represents a probability of your target variable, conditional on each value of the feature. In a way, it embodies the target variable in its encoded value. I have calculated mean encodings on the basis of following logical features I could think of:-
- item
- state
- store
- category
- department
- category & department
- store & item
- category & item
- department & item
- state & store
- state, store and category
- store, category and department

In [ ]:
data['iteam_sold_avg_mean'] = data.groupby('item_id')['sales'].transform('mean').astype(np.float16)
data['state_sold_avg_mean'] = data.groupby('state_id')['sales'].transform('mean').astype(np.float16)
data['store_sold_avg_mean'] = data.groupby('store_id')['sales'].transform('mean').astype(np.float16)
data['cat_sold_avg_mean'] = data.groupby('cat_id')['sales'].transform('mean').astype(np.float16)
data['dept_sold_avg_mean'] = data.groupby('dept_id')['sales'].transform('mean').astype(np.float16)
data['cat_dept_sold_avg_mean'] = data.groupby(['cat_id', 'dept_id'])['sales'].transform('mean').astype(np.float16)
data['store_item_sold_avg_mean'] = data.groupby(['store_id', 'item_id'])['sales'].transform('mean').astype(np.float16)
data['cat_item_sold_avg_mean'] = data.groupby(['cat_id', 'item_id'])['sales'].transform('mean').astype(np.float16)
data['dept_item_sold_avg_mean'] = data.groupby(['dept_id', 'item_id'])['sales'].transform('mean').astype(np.float16)
data['state_store_sold_avg_mean'] = data.groupby(['state_id', 'store_id'])['sales'].transform('mean').astype(np.float16)
data['state_store_cat_sold_avg_mean'] = data.groupby(['state_id', 'store_id', 'cat_id'])['sales'].transform('mean').astype(np.float16)
data['store_cat_dept_sold_avg_mean'] = data.groupby(['store_id', 'cat_id', 'dept_id'])['sales'].transform('mean').astype(np.float16)

In [ ]:
data['iteam_sold_avg_median'] = data.groupby('item_id')['sales'].transform('median').astype(np.float16)
data['state_sold_avg_median'] = data.groupby('state_id')['sales'].transform('median').astype(np.float16)
data['store_sold_avg_median'] = data.groupby('store_id')['sales'].transform('median').astype(np.float16)
data['cat_sold_avg_median'] = data.groupby('cat_id')['sales'].transform('median').astype(np.float16)
data['dept_sold_avg_median'] = data.groupby('dept_id')['sales'].transform('median').astype(np.float16)
data['cat_dept_sold_avg_median'] = data.groupby(['cat_id', 'dept_id'])['sales'].transform('median').astype(np.float16)
data['store_item_sold_avg_median'] = data.groupby(['store_id', 'item_id'])['sales'].transform('median').astype(np.float16)
data['cat_item_sold_avg_median'] = data.groupby(['cat_id', 'item_id'])['sales'].transform('median').astype(np.float16)
data['dept_item_sold_avg_median'] = data.groupby(['dept_id', 'item_id'])['sales'].transform('median').astype(np.float16)
data['state_store_sold_avg_median'] = data.groupby(['state_id', 'store_id'])['sales'].transform('median').astype(np.float16)
data['state_store_cat_sold_avg_median'] = data.groupby(['state_id', 'store_id', 'cat_id'])['sales'].transform('median').astype(np.float16)
data['store_cat_dept_sold_avg_median'] = data.groupby(['store_id', 'cat_id', 'dept_id'])['sales'].transform('median').astype(np.float16)

### 5.4 Rolling Window Stats

In [ ]:
data['rolling_mean_t7'] = data.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(7).mean()).astype(np.float16)
data['rolling_std_t7'] = data.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(7).std()).astype(np.float16)
data['rolling_std_t30'] = data.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(30).std()).astype(np.float16)
data['rolling_mean_t30'] = data.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(30).mean()).astype(np.float16)
data['rolling_mean_t90'] = data.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(90).mean()).astype(np.float16)
data['rolling_mean_t180'] = data.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(180).mean()).astype(np.float16)

### 5.5 Transform Price

In [ ]:
data['lag_price'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1))
data['price_change'] = (data['lag_price'] - data['sell_price']) / (data['lag_price']).astype(np.float16)
data['price_std'] = (data['sell_price'] - data['sell_price'].min()) / (1 + data['sell_price'].max() - data['sell_price'].min())
data['price_std'] = data['price_std'].astype(np.float16)
data.drop(['lag_price'], inplace = True, axis = 1)

### 5.6 Adding daytime

In [ ]:
data.drop(['weekday', 'wday', 'month', 'year', 'wm_yr_wk'], inplace=True, axis=1)

In [ ]:
data['date'] = pd.to_datetime(data['date'])
data['year'] = (data['date'].dt.year - 2010).astype(np.int16)
data['quarter'] = data['date'].dt.quarter.astype(np.int16)
data['month'] = data['date'].dt.month.astype(np.int16)
data['week'] = data['date'].dt.week.astype(np.int16)
data['day'] = data['date'].dt.day.astype(np.int16)
data['dayofweek'] = data['date'].dt.dayofweek.astype(np.int16)
data['d'] = data['d'].apply(lambda x: x.split('_')[1]).astype(np.int16)

<a id = 'F7'><h2>5.7 Saving the data</h2></a>

In [ ]:
data = reduce_mem_usage(data)

In [ ]:
data = data[data['d'] >= 180]
data.to_pickle(path + "cleaned_data.pkl")
data

<a id='pred'><h2>6. Modelling and Prediction</h2></a>

In [ ]:
# Original
params = {
        "metric": "rmse",
        "objective": "poisson",
        "alpha": 0.1,
        "lambda": 0.1,
        "seed": 42,
        "num_leaves": 100,
        "learning_rate": 0.075,
        "bagging_fraction": 0.75,
        "bagging_freq": 2,
        "colsample_bytree": 0.75,
    }

fit_params = {
        "num_boost_round": 2000,
        "early_stopping_rounds": 200,
        "verbose_eval": 100}

In [ ]:
features = data.columns.drop(['id', 'date', 'd', 'sales','revenue'])

In [ ]:
valid = data[(data['d'] >= 1914) & (data['d'] < 1942)][['id', 'd', 'sales']]
test = data[data['d'] >= 1942][['id', 'd', 'sales']]
eval_preds = test['sales']
valid_preds = valid['sales']

In [ ]:
# Training individual store

stores = data['store_id'].unique().tolist()
for store in stores:
    print("STORE TRAINING: ", store+1)
    df = data[data['store_id'] == store]

    #Split the data
    X_train, y_train = df[df['d'] < 1914].drop('sales', axis=1), df[df['d'] < 1914]['sales']
    X_valid, y_valid = df[(df['d'] >= 1914) & (df['d'] < 1942)].drop('sales', axis=1), df[(df['d'] >= 1914) & (df['d'] < 1942)]['sales']
    X_test = df[df['d'] >= 1942].drop('sales', axis=1)
    
    train_set = lgb.Dataset(X_train[features], y_train)
    valid_set = lgb.Dataset(X_valid[features], y_valid)

    #Train and validate
    model = lgb.train(params, train_set, valid_sets=[train_set, valid_set], **fit_params)
    valid_rmse = np.sqrt(mean_squared_error(model.predict(X_valid[features]), y_valid))
    eval_preds[X_test[features].index] = model.predict(X_test[features])

    del model, X_train, y_train, X_valid, y_valid
    gc.collect()
    
    print("\n")

In [ ]:
validation = sales[['id'] + ['d_' + str(i) for i in range(1914, 1942)]]
validation['id'] = pd.read_csv(path + 'sales_train_validation.csv').id
validation.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

In [ ]:
test['sales'] = eval_preds
predictions = test[['id', 'd', 'sales']]
predictions = pd.pivot(predictions, index='id', columns='d', values='sales').reset_index()
predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]
evaluation = sampleSubmission[['id']].merge(predictions, on = 'id')

In [ ]:
# Prepare the submission
submit = pd.concat([validation, evaluation]).reset_index(drop=True)
submit.to_csv('submission.csv', index=False)
submit[30490:]

<a id='con'><h2>7. Conclusion</h2></a>

With one model LightGBM and those features above, we can get 0.54928 private score and top 12 on M5 forecasting accuracy competition of Kaggle leader board.

![Capture.png](./Capture.png)

**Future Work**
- Focus more in feature engineering part
- Hyperparameter tunning
- Ensembling models Lasso, Ridge and LightGBM (certainly better score)
- Deploying categorical embedding
- Deploying LSTM model

**Reference**
- EDA reference

https://www.kaggle.com/anshuls235/time-series-forecasting-eda-fe-modelling